In [43]:
#一週間分の日付のリストを作成する
today = datetime.date.today()
date_list = [today + datetime.timedelta(days=day) for day in range(1,10)]
date_list = [date.strftime("%Y-%m-%d") for date in date_list]

print(date_list)



['2023-05-25', '2023-05-26', '2023-05-27', '2023-05-28', '2023-05-29', '2023-05-30', '2023-05-31', '2023-06-01', '2023-06-02']


In [24]:
'''5月20日(土)は0の付く日です。
過去3回分の0の付く日のデータを取得し、分析しました。
結果は以下の通りです。
・0の付く日の平均回転数は、0.0回転です。
・0の付く日の平均差枚数は、0.0枚です。
'''

'5月20日(土)は0の付く日です。\n過去3回分の0の付く日のデータを取得し、分析しました。\n結果は以下の通りです。\n・0の付く日の平均回転数は、0.0回転です。\n・0の付く日の平均差枚数は、0.0枚です。\n'

In [34]:
str(target_day_list).replace("'",'"')

'["2023-05-16", "2023-05-06", "2023-04-26"]'

In [92]:
from flask import Flask, render_template, request
from datetime import date, timedelta,datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

user_data = {}
user_data['tenpo-name'] = 'マルハン新宿東宝ビル店'
user_data['n-times'] = '3'
user_data['target-date'] = '2023-05-17'
print(user_data)
print(user_data['tenpo-name'],user_data['n-times'],user_data['target-date'])
target_day_list = []
number = 0
today = date.today()
target_number:int = str(user_data['target-date'][-1])
for i in range(int(user_data['n-times'])):
    while True:
        #print(str(date.today() - timedelta(days=number))[-1])
        if target_number == str(today - timedelta(days=number))[-1]:
            target_day = today - timedelta(days=number)
            print('取得日',target_day)
            target_day_str = target_day.strftime('%Y-%m-%d')
            target_day_list.append(target_day_str)
            number += 1
            break
        else: 
            pass
        number += 1
target_day_list.reverse()
concat_df_list = []
urls = []
for serch_date in target_day_list:
    search_url = url = f"https://ana-slo.com/{serch_date}-{user_data['tenpo-name']}-data/"
    urls.append(search_url)

with ThreadPoolExecutor(3) as executor:
    results = list(executor.map(requests.get, urls))
print(results)

for search_response,target_day in zip(results, target_day_list):
    soup = BeautifulSoup(search_response.text, "lxml")
    elem = soup.select('#all_data_block')
    dfs = pd.read_html(str(elem))
    for df in dfs:
        if '機種名' in list(df.columns):
            tmp_df = df
            tmp_df['日付'] = target_day
            #tmp_df['機種名'] = tmp_df['機種名'].map(removal_text)
            break
    concat_df_list.append(df)

concat_df = pd.concat(concat_df_list,axis=0)
for column_name in ['合成確率','BB確率','RB確率','台番号','ART確率']:
    try:
        concat_df = concat_df.drop([column_name],axis=1)
    except:
        pass

concat_df

{'tenpo-name': 'マルハン新宿東宝ビル店', 'n-times': '3', 'target-date': '2023-05-17'}
マルハン新宿東宝ビル店 3 2023-05-17
取得日 2023-05-17
取得日 2023-05-07
取得日 2023-04-27
[<Response [200]>, <Response [200]>, <Response [200]>]


,機種名,G数,差枚,BB,RB,日付
0,アイムジャグラーEX-TP,8571,-600,25,35,2023-04-27
1,アイムジャグラーEX-TP,7359,1800,34,21,2023-04-27
2,アイムジャグラーEX-TP,5261,-900,16,14,2023-04-27
3,アイムジャグラーEX-TP,8510,-500,29,27,2023-04-27
4,アイムジャグラーEX-TP,6577,-200,26,17,2023-04-27
...,...,...,...,...,...,...
521,マッピー,2958,100,11,4,2023-05-17
522,マッピー,3118,-900,11,6,2023-05-17
523,沖ドキ!DUO-30,6312,-2800,28,21,2023-05-17
524,沖ドキ!DUO-30,5278,-500,30,22,2023-05-17


,機種順位,機種名,総台数,G数,差枚
0,1,HEY！エリートサラリーマン鏡,102,629352,2900
1,2,この素晴らしい世界に祝福を！,18,78233,13000
2,3,とある科学の超電磁砲,15,65384,-1100
3,4,アイムジャグラーEX-TP,204,1438142,66200
4,5,アクエリオン ALL STARS,12,43527,-3600
5,6,アナザーゴッドハーデス‐解き放たれし槍撃ver.‐,36,184926,17600
6,7,アバサー,6,37194,4500
7,8,キングハナハナ-30,84,599238,5400
8,9,コードギアス反逆のルルーシュ3,6,37855,1000
9,10,ゴブリンスレイヤー,75,387417,-41400


In [99]:
kisyubetu_master_df = concat_df.groupby(['機種名']).sum()
kisyubetu_master_df['総台数'] = concat_df.groupby(['機種名']).size()
kisyubetu_master_df = kisyubetu_master_df.reset_index(drop=False).reset_index().rename(columns={'index': '機種順位','ゲーム数': 'G数'})
kisyubetu_master_df['機種順位'] = kisyubetu_master_df['機種順位'] + 1
kisyubetu_master_df[['機種順位','機種名','総台数','G数','差枚']]
kisyubetu_win_daissuu_list = []
kisyubetu_master_df_list = []
for kisyu_name in kisyubetu_master_df['機種名']:
    kisyu_df = concat_df.query('機種名 == @kisyu_name')
    kisyubetu_master_df_list.append(kisyu_df)
    kisyu_win_daisuu = len(kisyu_df[kisyu_df['差枚'] > 0])
    kisyubetu_win_daissuu_list.append(kisyu_win_daisuu)
kisyubetu_master_df['勝率'] = kisyubetu_win_daissuu_list
kisyubetu_master_df['勝率'] = kisyubetu_master_df['勝率'].astype(str)
kisyubetu_master_df['総台数'] = kisyubetu_master_df['総台数'].astype(int)
kisyubetu_master_df['平均G数'] = kisyubetu_master_df['G数'] / kisyubetu_master_df['総台数']
kisyubetu_master_df['平均G数'] = kisyubetu_master_df['平均G数'].astype(int)
kisyubetu_master_df = kisyubetu_master_df[kisyubetu_master_df['総台数'] >= 2 ]
kisyubetu_master_df['差枚'] = kisyubetu_master_df['差枚'].astype(int)
kisyubetu_master_df['平均差枚'] = kisyubetu_master_df['差枚'] / kisyubetu_master_df['総台数']
kisyubetu_master_df['平均差枚'] = kisyubetu_master_df['平均差枚'].astype(int) 
kisyubetu_master_df['総台数'] = kisyubetu_master_df['総台数'].astype(str) 
kisyubetu_master_df['勝率'] = kisyubetu_master_df['勝率'] + '/' + kisyubetu_master_df['総台数']
kisyubetu_master_df['勝率'] = kisyubetu_master_df['勝率'].map(lambda x : '(' + x + '台) ' + str(round(int(x.split('/')[0])/int(x.split('/')[1])*100,1))  + '%')
kisyubetu_master_df = kisyubetu_master_df[['機種順位','機種名','勝率','平均G数','平均差枚','差枚','G数','総台数']]
kisyubetu_master_df = kisyubetu_master_df.sort_values('平均差枚',ascending=False)
kisyubetu_master_df['機種順位'] = list(range(1,len(kisyubetu_master_df)+1))
kisyubetu_master_df['機種平均出率'] =(((kisyubetu_master_df['G数'] * 3) + kisyubetu_master_df['差枚']) / (kisyubetu_master_df['G数'] * 3) )*100
kisyubetu_master_df['機種平均出率'] = kisyubetu_master_df['機種平均出率'].map(lambda x : round(x,1))
kisyubetu_master_df['機種平均出率'] = kisyubetu_master_df['機種平均出率'].astype(str) + '%'
kisyubetu_master_df = kisyubetu_master_df.rename(columns={'G数': '合計G数','差枚': '合計差枚'})
kisyubetu_master_df

,機種順位,機種名,勝率,平均G数,平均差枚,合計差枚,合計G数,総台数,機種平均出率
19,1,バジリスク絆2,(25/27台) 92.6%,5509,2651,71600,148758,27,116.0%
35,2,戦姫絶唱シンフォギア 勇気の歌,(9/13台) 69.2%,6110,1646,21400,79430,13,109.0%
26,3,モンスターハンター:ワールドTM 黄金狩猟,(4/6台) 66.7%,6046,1400,8400,36279,6,107.7%
36,4,押忍!番長ZERO,(32/46台) 69.6%,4865,1319,60700,223797,46,109.0%
12,5,ディスクアップ2,(6/9台) 66.7%,5931,1133,10200,53384,9,106.4%
13,6,ニューゲッターマウス,(3/3台) 100.0%,5821,1066,3200,17464,3,106.1%
30,7,北斗の拳 宿命,(2/2台) 100.0%,4502,1050,2100,9005,2,107.8%
29,8,劇場版 魔法少女まどか☆マギカ[前編]始まりの物語/[後編]永遠の物語,(3/6台) 50.0%,3757,1016,6100,22542,6,109.0%
6,9,アバサー,(4/6台) 66.7%,6199,750,4500,37194,6,104.0%
44,10,甲鉄城のカバネリ,(29/51台) 56.9%,5237,731,37300,267113,51,104.7%


In [68]:
for search_response,target_day in zip(results, target_day_list):
    soup = BeautifulSoup(search_response.text, "lxml")
    elem = soup.select('#all_data_block')
    dfs = pd.read_html(str(elem))
    for df in dfs:
        if '機種名' in list(df.columns):
            tmp_df = df
            tmp_df['日付'] = target_day
            #tmp_df['機種名'] = tmp_df['機種名'].map(removal_text)
            break
    concat_df_list.append(df)

concat_df = pd.concat(concat_df_list,axis=0)
for column_name in ['合成確率','BB確率','RB確率','台番号','ART確率']:
    try:
        concat_df = concat_df.drop([column_name],axis=1)
    except:
        pass
concat_df = concat_df.drop(['機種名'],axis=1)
groupby_samai_game_mean_df = concat_df .groupby('日付').mean()
groupby_samai_game_mean_df.reset_index(inplace=True)
groupby_samai_game_mean_df['差枚'] = groupby_samai_game_mean_df['差枚'].astype(int)
groupby_samai_game_mean_df['G数'] = groupby_samai_game_mean_df['G数'].astype(int)
groupby_samai_game_mean_df['差枚'] = groupby_samai_game_mean_df['差枚'].astype(str)
groupby_samai_game_mean_df['G数'] = groupby_samai_game_mean_df['G数'].astype(str)
groupby_samai_game_mean_df['BB'] = groupby_samai_game_mean_df['BB'].astype(int)
groupby_samai_game_mean_df['RB'] = groupby_samai_game_mean_df['RB'].astype(int)
samai_list:list[int] = groupby_samai_game_mean_df['差枚'].tolist()
gamesuu_list:list[int] = groupby_samai_game_mean_df['G数'].tolist()
gamesuu_list

['1807', '1794', '1832', '1416']

In [69]:


record = [samai_list,gamesuu_list]
ave_tenpo_df = pd.DataFrame(record, columns=target_day_list,index=['平均差枚','平均G数'])
ave_tenpo_df[0:1]  =  ave_tenpo_df[0:1]  + '枚'
ave_tenpo_df[1:2]  =  ave_tenpo_df[1:2]  + 'G'
ave_tenpo_df


,2023-04-16,2023-04-26,2023-05-06,2023-05-16
平均差枚,-58枚,77枚,-48枚,64枚
平均G数,1807G,1794G,1832G,1416G


In [64]:
ave_tenpo_df

,2023-04-26,2023-05-06,2023-05-16
平均差枚,49枚,58枚,100枚
平均G数,4951G,5579G,5239G


In [56]:
ave_tenpo_df.

['4951', '5579', '5239']

In [8]:
touhou_df = df[df['店舗名'] == 'マルハン新宿東宝ビル店']
touhou_df[['日付', 'Nの日', '都道府県', '店舗名', '機種名', '台番号', '末尾', 'G数', '差枚',
       'BB', 'RB', 'BB確率', 'RB確率', '合成確率']]

,日付,Nの日,都道府県,店舗名,機種名,台番号,末尾,G数,差枚,BB,RB,BB確率,RB確率,合成確率
93054,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,917,7,2992.0,-800,8.0,7.0,1/374.0,1/427.4,1/199.5
93055,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,918,8,6365.0,100,23.0,21.0,1/276.7,1/303.1,1/144.7
93056,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,919,9,8552.0,-1300,25.0,30.0,1/342.1,1/285.1,1/155.5
93057,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,920,0,5093.0,1800,26.0,13.0,1/195.9,1/391.8,1/130.6
93058,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,921,1,6930.0,1600,31.0,23.0,1/223.5,1/301.3,1/128.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93541,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,ゼーガペイン2,995,5,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0
93542,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,閃乱カグラ BURST UP,996,6,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0
93543,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,ハイスクールD×D2 ハーレム王に俺はなる,997,7,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0
93544,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,ドリフターズ,998,8,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0


In [ ]:
touhou_df = df[df['店舗名'] == 'マルハン新宿東宝ビル店']
touhou_df[['日付', 'Nの日', '都道府県', '店舗名', '機種名', '台番号', '末尾', 'G数', '差枚',
       'BB', 'RB', 'BB確率', 'RB確率', '合成確率']]


,都道府県,店舗名,URL,url_店舗名
0,北海道,BIG大王東店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,big大王東店
1,北海道,CREV伊達店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,crev伊達店
2,北海道,EBH,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,ebh
3,北海道,INDY函館,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,indy函館
4,北海道,KEIO湯川店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,keio湯川店
...,...,...,...,...
3180,沖縄県,マルシンV3,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,マルシンv3
3181,沖縄県,マルシンV4,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,マルシンv4
3182,沖縄県,ラッキー高原店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,ラッキー高原店
3183,沖縄県,ルクソール,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,ルクソール


In [1]:
import pandas as pd



In [3]:
tenpo_url_df:pd.DataFrame = pd.read_csv('csv/tenpo_url_flask_web_site.csv')
group_name_list:list[str] ='''123,BBステーション,DAS,Dステーション,MGM,PIA,やすだ,ウエスタン,エスパス,\
    エンジェル,オゼック,オリエンタルパサージュ,オーパス・ワン,カレイド,キコーナ,\
    キューデンアネックス,グランパ,コンサートホール,ゴードン,ジャラン,\
    ジャンジャンマールゴット,デルパラ,ドキわくランド,ニラク,パラッツォ,\
    パーラースーパーセブン,パーラーフィオーレ,ヒノマル,ヒロキ,ビックディッパー,\
    ピーアーク,フルハウス,プレゴ,ベガスベガス,マルハン,ミカド,ミリオン,ガイア,\
    メッセ,国際センター,UNO,楽園,オーシャン,金時'''.split(',')
others_extract_tokyo_tenpo_url_df = extract_tokyo_tenpo_url_df = tenpo_url_df[tenpo_url_df['都道府県'] == '東京都']
extract_tokyo_tenpo_url_df = extract_tokyo_tenpo_url_df.sort_values('店舗名')
extract_tokyo_tenpo_url_df

group_name_count_dict:dict[str] = {}
web_group_name_list:list[str] = []
for group_name in group_name_list:
    extract_groupname_df = extract_tokyo_tenpo_url_df[extract_tokyo_tenpo_url_df['店舗名'].str.contains(group_name)]
    #display(extract_groupname_df)
    group_name_count_dict[group_name] = len(extract_groupname_df)
    #break
    #print(group_name)
    tmp_list = []
    #print(group_name)
    for tenpo_name in extract_groupname_df['店舗名']:
        tmp_list.append(tenpo_name)
        target = extract_tokyo_tenpo_url_df.index[(extract_tokyo_tenpo_url_df['店舗名'] == tenpo_name)]
    web_group_name_list.append(tmp_list)


sorted_group_name_count_dict = sorted(group_name_count_dict.items(), key = lambda fruit : fruit[1])
print(type(sorted_group_name_count_dict))
# <class 'list'>
sorted_group_name_count_dict.reverse()
sorted_group_name_count_dict = dict(sorted_group_name_count_dict)
sorted_group_name_count_dict

group_name_count_dict:dict[str] = {}
web_group_name_list:list[str] = []
for group_name in sorted_group_name_count_dict:
    extract_groupname_df = extract_tokyo_tenpo_url_df[extract_tokyo_tenpo_url_df['店舗名'].str.contains(group_name)]
    #display(extract_groupname_df)
    group_name_count_dict[group_name] = len(extract_groupname_df)
    #break
    #print(group_name)
    tmp_list = []
    #print(group_name)
    for tenpo_name in extract_groupname_df['店舗名']:
        tmp_list.append(tenpo_name)
        target = extract_tokyo_tenpo_url_df.index[(extract_tokyo_tenpo_url_df['店舗名'] == tenpo_name)]
    web_group_name_list.append(tmp_list)
    

others_extract_tokyo_tenpo_url_df = extract_tokyo_tenpo_url_df 
for group_name in sorted_group_name_count_dict:
    extract_groupname_df = extract_tokyo_tenpo_url_df[extract_tokyo_tenpo_url_df['店舗名'].str.contains(group_name)]
    for tenpo_name in extract_groupname_df['店舗名']:
        target = extract_tokyo_tenpo_url_df.index[(extract_tokyo_tenpo_url_df['店舗名'] == tenpo_name)]
        others_extract_tokyo_tenpo_url_df = others_extract_tokyo_tenpo_url_df.drop(target)
web_group_name_list.append(list(others_extract_tokyo_tenpo_url_df['店舗名'].unique()))
sorted_group_name_count_dict['その他'] = len(others_extract_tokyo_tenpo_url_df)

<class 'list'>


In [6]:
group_num_list = list(sorted_group_name_count_dict.values())
display_group_name_list = list(sorted_group_name_count_dict.keys())
for group_name_list,display_group_name_list,group_num in zip(web_group_name_list,display_group_name_list,group_num_list):
    print(display_group_name_list,group_num)
    for tenpo_name in group_name_list:
        print('・',tenpo_name)
    #break

マルハン 13
・ マルハン亀有店
・ マルハン八王子四谷店
・ マルハン大山店
・ マルハン小岩店
・ マルハン新宿東宝ビル店
・ マルハン日野店
・ マルハン昭島店
・ マルハン東大和店
・ マルハン池袋 SLOT BASE
・ マルハン池袋店
・ マルハン蒲田駅東店
・ マルハン青梅新町店
・ マルハン鹿浜店
ヒノマル 11
・ ヒノマル下丸子店
・ ヒノマル下井草店
・ ヒノマル中目黒店
・ ヒノマル広尾店
・ ヒノマル江古田店
・ ヒノマル狛江店
・ ヒノマル用賀店
・ ヒノマル西小山店
・ ヒノマル都立大１号店
・ ヒノマル錦糸町店スロット館
・ ヒノマル駒沢店
エスパス 11
・ エスパス1300新小岩北口駅前店
・ エスパス日拓上野新館
・ エスパス日拓上野本館
・ エスパス日拓新大久保駅前店
・ エスパス日拓新宿歌舞伎町店
・ エスパス日拓渋谷本館
・ エスパス日拓渋谷駅前新館
・ エスパス日拓秋葉原駅前店
・ エスパス日拓西武新宿駅前店
・ エスパス日拓赤坂見附駅前新館
・ エスパス日拓高田馬場本店
楽園 8
・ 楽園アメ横店
・ 楽園大山店
・ 楽園池袋店
・ 楽園渋谷道玄坂店
・ 楽園渋谷駅前店
・ 楽園町田店
・ 楽園立川店
・ 楽園蒲田店
PIA 7
・ PIA上野
・ PIA五反野
・ PIA八王子
・ PIA大森
・ PIA池袋
・ PIA町田新館
・ PIA雑色新館
UNO 6
・ 三ノ輪UNO
・ 仙川UNO
・ 新橋UNO
・ 有楽町UNO
・ 清瀬UNO
・ 秋葉原UNO
コンサートホール 6
・ コンサートホール北千住
・ コンサートホール志村
・ コンサートホール成増2
・ コンサートホール成増スロット館
・ コンサートホール荒川
・ コンサートホール調布
やすだ 6
・ やすだひばりヶ丘店
・ やすだ仲宿店
・ やすだ大山北口5号店
・ やすだ東池袋9号店
・ やすだ清水町店
・ やすだ西池袋6号店
ミリオン 5
・ ミリオン平和台16号店
・ ミリオン成増7号店
・ ミリオン東武練馬13号店
・ ミリオン石神井公園店スロット館
・ ミリオン高円寺22号店
プレゴ 5
・ スマートプレゴ池袋
・ プレゴEX池袋南口店
・ プレゴひばりが丘店
・ プレゴ立川店
・ プレゴ自由が丘店
ニラク 5
・ ニラク上北台店
・ ニラ

In [4]:
from PIL  import ImageDraw , ImageFont , Image
def get_concat_h_multi_resize(im_list, resample=Image.BICUBIC):
    min_height = min(im.height for im in im_list)
    im_list_resize = [im.resize((int(im.width * min_height / im.height), min_height),resample=resample)
                    for im in im_list]
    total_width = sum(im.width for im in im_list_resize)
    dst = Image.new('RGB', (total_width, min_height))
    pos_x = 0
    for im in im_list_resize:
        dst.paste(im, (pos_x, 0))
        pos_x += im.width
    return dst

def get_concat_v_multi_resize(im_list, resample=Image.BICUBIC):
    min_width = min(im.width for im in im_list)
    im_list_resize = [im.resize((min_width, int(im.height * min_width / im.width)),resample=resample)
                    for im in im_list]
    total_height = sum(im.height for im in im_list_resize)
    dst = Image.new('RGB', (min_width, total_height))
    pos_y = 0
    for im in im_list_resize:
        dst.paste(im, (0, pos_y))
        pos_y += im.height
    return dst

#テーブルタイプの画像出力
def create_df_cell_image(_df,image_name):
    global create_df_cell_image_path
    width_concat_lists = []
    font = ImageFont.truetype('font/MochiyPopOne-OTF-ExtraBold.otf', 18)
    df_columns_list = list(_df.columns)
    for column_number in range(len(_df.columns)):
        height_concat_lists = []
        #print(column_number)
        print(df_columns_list[column_number])
        if df_columns_list[column_number] == '機種名':
            cell_width = 380
        elif df_columns_list[column_number] == '店舗名':
            cell_width = 300
        elif df_columns_list[column_number] == '店舗平均差枚':
            cell_width = 150
        elif df_columns_list[column_number] == '店舗平均G数':
            cell_width = 150
        elif df_columns_list[column_number] == '勝率':
            cell_width = 200
        elif df_columns_list[column_number] == 'データ':
            cell_width = 200
        else:
            cell_width = 100
        cell_height = 40
        im = Image.new('RGB', (cell_width, cell_height), (139, 0, 206))  # イメージオブジェクトの生成(黒のベタ画像)
        draw = ImageDraw.Draw(im)  # Drawオブジェクトを生成  
        # フォントの指定(メイリオ48pt)
        draw.multiline_text((cell_width/2, 20), df_columns_list[column_number], fill=(255,255,255), font=font, align ="center",anchor="mm") # 文字の描画
        w, h = im.size
        draw.rectangle((0, 0, w-1, h-1), outline = (255,255,255))
        height_concat_lists.append(im)
        for index_number ,(i,record) in enumerate(_df.iterrows()):
            if (index_number + 1 ) %  2 != 0:
                im = Image.new('RGB', (cell_width, cell_height), (255, 255, 255))  # イメージオブジェクトの生成(黒のベタ画像)
            else:
                im = Image.new('RGB', (cell_width, cell_height), (202, 168, 255))  # イメージオブジェクトの生成(黒のベタ画像)
            draw = ImageDraw.Draw(im)  # Drawオブジェクトを生成  
            
            draw.multiline_text((cell_width/2,10), f'{record[column_number]}', fill=(0,0,0), font=font,anchor="ma") 
            w, h = im.size
            
            if df_columns_list[column_number] == '店舗平均差枚':
                samai = record[column_number]
                if samai > 0:
                    samai_bunbo = 400
                    samai_percet = samai / samai_bunbo
                    draw.rectangle([(30, 0), (30+ samai_percet * 100, 40)], fill=(124, 233, 255))
                else :
                    samai_bunbo = 500
                    samai_percet = samai / samai_bunbo
                    draw.rectangle([(30, 0),  (30+ samai_percet * 100, 40)], fill=(255, 0,0))
                draw.rectangle([(29, 0), (30, 40)], fill=(255, 255, 255)) 
                
            if df_columns_list[column_number] == '差枚' :
                samai = record[column_number]
                if samai > 0:
                    samai_bunbo = 50000
                    samai_percet = samai / samai_bunbo
                    draw.rectangle([(30, 0), (30+ samai_percet * 100, 40)], fill=(124, 233, 255))
                else :
                    samai_bunbo = 50000
                    samai_percet = samai / samai_bunbo
                    draw.rectangle([(30, 0),  (30+ samai_percet * 50, 40)], fill=(255, 0,0))
                draw.rectangle([(29, 0), (30, 40)], fill=(255, 255, 255))  
                
            if df_columns_list[column_number] == 'G数' or df_columns_list[column_number] == 'ゲーム数' or df_columns_list[column_number] == '店舗平均G数':
                gamesuu = record[column_number]
                samai_bunbo = 6000
                samai_percet = gamesuu / samai_bunbo
                #print(samai_percet)
                draw.rectangle([(0, 0), (samai_percet * 100, 40)], fill=(0, 255, 206))

            if df_columns_list[column_number] ==  'BB' or df_columns_list[column_number] ==  'RB' or df_columns_list[column_number] ==  'ART' :
                atari_kaisuu = record[column_number]
                bunbo = 35
                percet = atari_kaisuu / bunbo
                #print(percet)
                draw.rectangle([(0, 0), (percet * 100, 40)], fill=(255, 193, 133))
                
            if df_columns_list[column_number] ==  '総台数'  :
                atari_kaisuu = int(record[column_number])
                bunbo = 20
                percet = atari_kaisuu / bunbo
                #print(percet)
                draw.rectangle([(0, 0), (percet * 100, 40)], fill=(255, 193, 133))
            if df_columns_list[column_number] ==  '台数'  :
                atari_kaisuu = int(record[column_number])
                bunbo = 500
                percet = atari_kaisuu / bunbo
                #print(percet)
                draw.rectangle([(0, 0), (percet * 100, 40)], fill=(255, 193, 133))
                
            if df_columns_list[column_number] ==  '勝率'  :
                atari_kaisuu = float(record[column_number].split(')')[1].replace('%','').replace(' ',''))
                #print(atari_kaisuu)
                draw.rectangle([(0, 0), ((atari_kaisuu/100)*200, 40)], fill=(251, 244, 0))
            
            else:
                pass
                
            draw.multiline_text((cell_width/2,10), f'{record[column_number]}', fill=(0,0,0), font=font,anchor="ma")
            draw.rectangle((0, 0, w-1, h-1), outline = (0,0,0))
            height_concat_lists.append(im)
            
        #break
        concat_image_path  = rf"image/temp_image/complted_cell_{column_number}.png"
        get_concat_v_multi_resize(height_concat_lists).save(concat_image_path)
        concat_im = Image.open(concat_image_path)
        width_concat_lists.append(concat_im)
    create_df_cell_image_path = rf"image/temp_image/temp_complted_df_image_cell_{image_name}.png"
    get_concat_h_multi_resize(width_concat_lists).save(create_df_cell_image_path)
    return create_df_cell_image_path


#### 機種情報サイトからスロット台の機種情報をスクレイピングする

In [5]:
import urllib
from bs4 import BeautifulSoup
import re
import pandas as pd
import codecs
import requests
import time
import urllib.request as req


In [39]:
url = 'https://p-town.dmm.com/machines/search?machine_type=1'
response = req.urlopen(url) #データを取得
soup = BeautifulSoup(response, 'html.parser')
card_lists = soup.find_all(class_='unit')

In [50]:
len(card_lists)

20

In [51]:
card.text

'\n\n\n\n\n\n\n\n\n\nパチスロ 戦国†恋姫\n藤商事\n\n                                                                                            機械割: 掲載準備中\n                                                                                    \n通常時はおもにレア役などを契機にボーナス（AT）当選を目指すゲーム性。\r\nボーナスはBIGとREGの2種類で、REGは約100枚を獲得できる。BIG…\n\n\n\n\n導入開始日:2023/07/18予定\nパチスロ\n\n\n'

In [61]:
url = 'https://p-town.dmm.com/machines/search?machine_type=1'
response = req.urlopen(url) #データを取得
soup = BeautifulSoup(response, 'html.parser')
card_lists = soup.find_all(class_='unit')
concat_df = pd.DataFrame(index=[], columns=[])
for i,card in enumerate(card_lists):
    print(card.text)
    card_url = card.find('a').get('href')
    card_url = 'https://p-town.dmm.com' + card_url
    print(card_url)
    response = req.urlopen(card_url) #データを取得
    card_soup = BeautifulSoup(response, 'html.parser')
    #print(soup)
    kisyu_name =  card_soup.find(class_='titleruby').find(class_='title').text.split('\n')[1].replace(' ','')
    pd.read_html(card_soup.prettify())[0].T
    thumbnail_image = card_soup.find(class_='image').find('img').get('src')
    thumbnail_image_url:str = 'https://p-town.dmm.com' + thumbnail_image

    sub_image = card_soup.find(class_='subimage').find('img').get('src')
    sub_image_url = 'https://p-town.dmm.com' + thumbnail_image
    print(sub_image_url)
    
    overview_df = pd.read_html(card_soup.prettify())[0].T
    overview_df.columns = overview_df.iloc[0]
    overview_df = overview_df.drop(overview_df.index[0])
    overview_df = overview_df.reset_index(drop=True)
    overview_df
    df = pd.DataFrame([[kisyu_name,thumbnail_image_url,sub_image_url]],columns=['機種名', 'thumbnail_image_url', 'sub_image_url'])
    merge_df = pd.concat([df, overview_df], axis=1)
    merge_df
    concat_df = pd.concat([concat_df, merge_df], axis=0)

    if i > 10:
        break
    













スマスロキン肉マン～7人の悪魔超人編～
セブンリーグ

                                                                                            機械割: 97.9% 〜 114.9%
                                                                                    
人気の高い「7人の悪魔超人編」をモチーフにしたパチスロ。
ATはつねに純増約6.1枚/Gで、悪魔超人を7人倒すと突入する「魔界ループ」が、出玉増加…




導入開始日:2023/08上旬予定
パチスロ



https://p-town.dmm.com/machines/4378
https://p-town.dmm.com/upload/machines/4378/image.jpg?t=1684230128










スーパービンゴネオクラシック-30
ベルコ

                                                                                            機械割: 98.0% 〜 106.0%
                                                                                    




導入開始日:2023/08予定
パチスロ



https://p-town.dmm.com/machines/4387
https://p-town.dmm.com/upload/machines/4387/image.jpg?t=1686034895










スーパービンゴネオクラシック
ベルコ

                                                                                            機械割: 98.0% 〜 106.0%
                                                    

AttributeError: 'NoneType' object has no attribute 'find'

In [64]:
card_soup

<!DOCTYPE html>

<html class="g-html" id="to-pagetop" style="">
<head>
<title>SナイツYTCC（パチスロ）設定判別・天井・ゾーン・解析・打ち方・ヤメ時</title>
<meta content="【最新攻略情報 随時更新】SナイツYTCCのパチスロ機種情報。DMMぱちタウンでは、設定判別要素、天井や立ち回りポイント、ヤメ時、打ち方、全設定の機械割、小役確率などの解析情報が充実！さらにランキングから来店レポート、店舗検索まで無料で公開中！" name="description"/>
<meta content="shovf64C2dkJGZaHvDsUdGvwtBX8RJsbIf14NAg2" name="csrf-token"/>
<link as="style" href="https://navismithapis-cdn.com/css/pc/min.css?v=6f179c71a9c5f8574e8657c628528d02e73bfd97" rel="preload"/>
<link href="https://navismithapis-cdn.com/css/pc/min.css?v=6f179c71a9c5f8574e8657c628528d02e73bfd97" rel="stylesheet"/><script crossorigin="anonymous" data-env="prod" data-site-type="general" data-version="v2" data-view-type="pc" defer="" id="naviapi-pigeon" src="https://navismithapis-cdn.com/js/pigeon.js?v=6f179c71a9c5f8574e8657c628528d02e73bfd97"></script>
<script defer="defer" src="https://cdj.dap.dmm.com/dmm-c-sdk.js" type="text/javascript"></script>
<script crossorigin="anonymous" defer="" src="https:/

In [60]:
card_soup.find(class_='titleruby').find(class_='title').text.split('\n')[1].replace(' ','')

'スマスロキン肉マン～7人の悪魔超人編～'

In [21]:
thumbnail_image = soup.find(class_='image').find('img').get('src')
image = 'https://p-town.dmm.com' + thumbnail_image
print(image)

https://p-town.dmm.com/upload/machines/4378/image.jpg?t=1684230128
